## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [52]:
from sqlalchemy import create_engine, text, MetaData,select,desc,func
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [53]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [54]:
### Imprima os três primeiros registros da tabela tracks
albums = Base.classes.albums
res = session.scalars(select(albums).limit(3)).all()
for album in res:
    print(album.AlbumId,album.Title)

1 For Those About To Rock We Salute You
2 Balls to the Wall
3 Restless and Wild


In [55]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
tracks = Base.classes.tracks
albums = Base.classes.albums
query_tracks = session.execute(select(tracks).limit(20)).scalars().all()
i = 0
lista_name_album = []
lista_track_name = []
for track in query_tracks:
    query_album = session.execute(select(albums).where(albums.AlbumId == track.AlbumId)).scalars().all()
    lista_track_name.append(track.Name)
    for album in query_album:
        lista_name_album.append(album.Title)
    
for i in range(len(lista_track_name)):
    print(f'Faixa: {lista_track_name[i]} --- Album: {lista_name_album[i]}')



Faixa: For Those About To Rock (We Salute You) --- Album: For Those About To Rock We Salute You
Faixa: Balls to the Wall --- Album: Balls to the Wall
Faixa: Fast As a Shark --- Album: Restless and Wild
Faixa: Restless and Wild --- Album: Restless and Wild
Faixa: Princess of the Dawn --- Album: Restless and Wild
Faixa: Put The Finger On You --- Album: For Those About To Rock We Salute You
Faixa: Let's Get It Up --- Album: For Those About To Rock We Salute You
Faixa: Inject The Venom --- Album: For Those About To Rock We Salute You
Faixa: Snowballed --- Album: For Those About To Rock We Salute You
Faixa: Evil Walks --- Album: For Those About To Rock We Salute You
Faixa: C.O.D. --- Album: For Those About To Rock We Salute You
Faixa: Breaking The Rules --- Album: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives --- Album: For Those About To Rock We Salute You
Faixa: Spellbound --- Album: For Those About To Rock We Salute You
Faixa: Go Down --- Album: Let There Be Rock


In [56]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
tracks = Base.classes.tracks
invoice_items = Base.classes.invoice_items 
query_invoice = session.execute(select(invoice_items).limit(10)).scalars().all()
# for item in query_invoice:
#     for column in item.__table__.columns:
#         nome = column.name
#         valor = getattr(item, nome)
#         print(f"{nome}: {valor}")
#     print('-' * 30)


for invoice in query_invoice:
    query_faixa = session.execute(select(tracks).where(tracks.TrackId == invoice.TrackId)).scalars().first()
    print(f'Faixa: {query_faixa.Name} --- Quantidade: {invoice.Quantity}')
    

    
    
    

Faixa: Balls to the Wall --- Quantidade: 1
Faixa: Restless and Wild --- Quantidade: 1
Faixa: Put The Finger On You --- Quantidade: 1
Faixa: Inject The Venom --- Quantidade: 1
Faixa: Evil Walks --- Quantidade: 1
Faixa: Breaking The Rules --- Quantidade: 1
Faixa: Dog Eat Dog --- Quantidade: 1
Faixa: Overdose --- Quantidade: 1
Faixa: Love In An Elevator --- Quantidade: 1
Faixa: Janie's Got A Gun --- Quantidade: 1


In [57]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
query_desc_quantity = session.execute(select(invoice_items)).scalars().all()
i = 0
dict_quantidade = {}
for invoice in query_desc_quantity:
    dict_quantidade[invoice.TrackId] = dict_quantidade.get(invoice.TrackId, 0) + 1

dict_ordenado = dict(sorted(dict_quantidade.items(), key=lambda item: item[1], reverse=True))

for track_id, quantidade in dict_ordenado.items():
    if i < 10:
        query_faixa = session.execute(select(tracks).where(tracks.TrackId == track_id)).scalars().first()
        print(f"Faixa: {query_faixa.Name} --- Quantidade de vendas: {quantidade}")
    i += 1
    
    


Faixa: Balls to the Wall --- Quantidade de vendas: 2
Faixa: Inject The Venom --- Quantidade de vendas: 2
Faixa: Overdose --- Quantidade de vendas: 2
Faixa: Deuces Are Wild --- Quantidade de vendas: 2
Faixa: Not The Doctor --- Quantidade de vendas: 2
Faixa: Por Causa De Você --- Quantidade de vendas: 2
Faixa: Welcome Home (Sanitarium) --- Quantidade de vendas: 2
Faixa: Cornucopia --- Quantidade de vendas: 2
Faixa: Bowels Of The Devil --- Quantidade de vendas: 2
Faixa: When My Left Eye Jumps --- Quantidade de vendas: 2


In [59]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
invoice_items = Base.classes.invoice_items
tracks = Base.classes.tracks
albums = Base.classes.albums
artists = Base.classes.artists

query = (
    select(artists.Name, func.sum(invoice_items.Quantity).label('total_vendas'))
    .join(albums, albums.ArtistId == artists.ArtistId)
    .join(tracks, tracks.AlbumId == albums.AlbumId)
    .join(invoice_items, invoice_items.TrackId == tracks.TrackId)
    .group_by(artists.ArtistId)
    .order_by(desc('total_vendas'))
    .limit(10)
)

resultado = session.execute(query).all()
for nome, total in resultado:
    print(f"{nome}: {total}")

Iron Maiden: 140
U2: 107
Metallica: 91
Led Zeppelin: 87
Os Paralamas Do Sucesso: 45
Deep Purple: 44
Faith No More: 42
Lost: 41
Eric Clapton: 40
R.E.M.: 39
